In [1]:
import pdfplumber
import pprint
import os
import re
import csv
import pandas as pd
from pathlib import Path

In [2]:
# Setting directory
dbs_source_dir = Path("/Users/jeromeko/Desktop/2020_Bank_Statements/DBS")
uob_source_dir = Path("/Users/jeromeko/Desktop/2020_Bank_Statements/UOB")
dbs_pdf_file = "DBS_202012.pdf"
uob_pdf_file = "UOB_202008.pdf"
dest_csv = Path("/Users/jeromeko/Desktop/2020_Bank_Statements")

# Processing DBS first page

In [3]:
# Reads PDF's first page and extracts all txns after specified header (impt that it's fixed throughout all PDFs)
with pdfplumber.open(dbs_source_dir / dbs_pdf_file) as pdf:
    first_page = pdf.pages[0]
    first_page_text = first_page.extract_text()
    first_page_txns_raw = first_page_text.partition("NEW TRANSACTIONS JEROME KO JIA JIN")[2]
    print(first_page_txns_raw)

 R
o. 
12 NOV LAZADA SINGAPORE (PAYM              13.39 C
13 NOV GRAB* BF5684F0017318B3              40.00
15 NOV FAIRPRICE XTRA-AMK              26.00
15 NOV AMAZON SG             177.03
15 NOV STARBUCKS@AMK HUB(AMK)              10.43
15 NOV AMAZON SG             134.59
16 NOV UNIQLO CITY SQUARE              67.70
16 NOV DECATHLON SG LAB              66.00
16 NOV BANANA LEAF              55.30
20 NOV ZaloraSG              19.51CR
20 NOV TENDON KOHAKU              35.31
20 NOV GRAB* S-26872624-8-462              11.00
20 NOV MCDONALD'S (HG)               3.20
20 NOV MCDONALD'S (HG)              20.15
22 NOV MCDONALD'S (SHHG)               7.20
23 NOV NTUC FP-HOUGANG A              11.65
23 NOV ESSO-CHEERS BY FP              45.00
27 NOV ZALORASG              95.73
27 NOV LAZADA SINGAPORE              55.20
27 NOV OMOTE              76.03
27 NOV SHOPEE - SINGAPORE               3.84
27 NOV SHOPEE SINGAPORE               4.25
28 NOV GRAB* A96461585C435436              10.00


In [4]:
# Looks at whitespace characters in text
print(repr(first_page_txns_raw))

" R\no. \n12 NOV LAZADA SINGAPORE (PAYM              13.39 C\n13 NOV GRAB* BF5684F0017318B3              40.00\n15 NOV FAIRPRICE XTRA-AMK              26.00\n15 NOV AMAZON SG             177.03\n15 NOV STARBUCKS@AMK HUB(AMK)              10.43\n15 NOV AMAZON SG             134.59\n16 NOV UNIQLO CITY SQUARE              67.70\n16 NOV DECATHLON SG LAB              66.00\n16 NOV BANANA LEAF              55.30\n20 NOV ZaloraSG              19.51CR\n20 NOV TENDON KOHAKU              35.31\n20 NOV GRAB* S-26872624-8-462              11.00\n20 NOV MCDONALD'S (HG)               3.20\n20 NOV MCDONALD'S (HG)              20.15\n22 NOV MCDONALD'S (SHHG)               7.20\n23 NOV NTUC FP-HOUGANG A              11.65\n23 NOV ESSO-CHEERS BY FP              45.00\n27 NOV ZALORASG              95.73\n27 NOV LAZADA SINGAPORE              55.20\n27 NOV OMOTE              76.03\n27 NOV SHOPEE - SINGAPORE               3.84\n27 NOV SHOPEE SINGAPORE               4.25\n28 NOV GRAB* A96461585C435436       

In [5]:
# Split by new lines
first_page_txns_raw_nl = first_page_txns_raw.split("\n")
pprint.pprint(first_page_txns_raw_nl)

[' R',
 'o. ',
 '12 NOV LAZADA SINGAPORE (PAYM              13.39 C',
 '13 NOV GRAB* BF5684F0017318B3              40.00',
 '15 NOV FAIRPRICE XTRA-AMK              26.00',
 '15 NOV AMAZON SG             177.03',
 '15 NOV STARBUCKS@AMK HUB(AMK)              10.43',
 '15 NOV AMAZON SG             134.59',
 '16 NOV UNIQLO CITY SQUARE              67.70',
 '16 NOV DECATHLON SG LAB              66.00',
 '16 NOV BANANA LEAF              55.30',
 '20 NOV ZaloraSG              19.51CR',
 '20 NOV TENDON KOHAKU              35.31',
 '20 NOV GRAB* S-26872624-8-462              11.00',
 "20 NOV MCDONALD'S (HG)               3.20",
 "20 NOV MCDONALD'S (HG)              20.15",
 "22 NOV MCDONALD'S (SHHG)               7.20",
 '23 NOV NTUC FP-HOUGANG A              11.65',
 '23 NOV ESSO-CHEERS BY FP              45.00',
 '27 NOV ZALORASG              95.73',
 '27 NOV LAZADA SINGAPORE              55.20',
 '27 NOV OMOTE              76.03',
 '27 NOV SHOPEE - SINGAPORE               3.84',
 '27 NOV SHO

In [6]:
print(repr(first_page_txns_raw_nl))

[' R', 'o. ', '12 NOV LAZADA SINGAPORE (PAYM              13.39 C', '13 NOV GRAB* BF5684F0017318B3              40.00', '15 NOV FAIRPRICE XTRA-AMK              26.00', '15 NOV AMAZON SG             177.03', '15 NOV STARBUCKS@AMK HUB(AMK)              10.43', '15 NOV AMAZON SG             134.59', '16 NOV UNIQLO CITY SQUARE              67.70', '16 NOV DECATHLON SG LAB              66.00', '16 NOV BANANA LEAF              55.30', '20 NOV ZaloraSG              19.51CR', '20 NOV TENDON KOHAKU              35.31', '20 NOV GRAB* S-26872624-8-462              11.00', "20 NOV MCDONALD'S (HG)               3.20", "20 NOV MCDONALD'S (HG)              20.15", "22 NOV MCDONALD'S (SHHG)               7.20", '23 NOV NTUC FP-HOUGANG A              11.65', '23 NOV ESSO-CHEERS BY FP              45.00', '27 NOV ZALORASG              95.73', '27 NOV LAZADA SINGAPORE              55.20', '27 NOV OMOTE              76.03', '27 NOV SHOPEE - SINGAPORE               3.84', '27 NOV SHOPEE SINGAPORE          

In [7]:
# Further split according to spaces
first_page_txns_raw_categorized = [txn.split() for txn in first_page_txns_raw_nl]
pprint.pprint(first_page_txns_raw_categorized)

[['R'],
 ['o.'],
 ['12', 'NOV', 'LAZADA', 'SINGAPORE', '(PAYM', '13.39', 'C'],
 ['13', 'NOV', 'GRAB*', 'BF5684F0017318B3', '40.00'],
 ['15', 'NOV', 'FAIRPRICE', 'XTRA-AMK', '26.00'],
 ['15', 'NOV', 'AMAZON', 'SG', '177.03'],
 ['15', 'NOV', 'STARBUCKS@AMK', 'HUB(AMK)', '10.43'],
 ['15', 'NOV', 'AMAZON', 'SG', '134.59'],
 ['16', 'NOV', 'UNIQLO', 'CITY', 'SQUARE', '67.70'],
 ['16', 'NOV', 'DECATHLON', 'SG', 'LAB', '66.00'],
 ['16', 'NOV', 'BANANA', 'LEAF', '55.30'],
 ['20', 'NOV', 'ZaloraSG', '19.51CR'],
 ['20', 'NOV', 'TENDON', 'KOHAKU', '35.31'],
 ['20', 'NOV', 'GRAB*', 'S-26872624-8-462', '11.00'],
 ['20', 'NOV', "MCDONALD'S", '(HG)', '3.20'],
 ['20', 'NOV', "MCDONALD'S", '(HG)', '20.15'],
 ['22', 'NOV', "MCDONALD'S", '(SHHG)', '7.20'],
 ['23', 'NOV', 'NTUC', 'FP-HOUGANG', 'A', '11.65'],
 ['23', 'NOV', 'ESSO-CHEERS', 'BY', 'FP', '45.00'],
 ['27', 'NOV', 'ZALORASG', '95.73'],
 ['27', 'NOV', 'LAZADA', 'SINGAPORE', '55.20'],
 ['27', 'NOV', 'OMOTE', '76.03'],
 ['27', 'NOV', 'SHOPEE', '-', 

In [8]:
[txn for txn in first_page_txns_raw_categorized if len(txn) >= 4]

[['12', 'NOV', 'LAZADA', 'SINGAPORE', '(PAYM', '13.39', 'C'],
 ['13', 'NOV', 'GRAB*', 'BF5684F0017318B3', '40.00'],
 ['15', 'NOV', 'FAIRPRICE', 'XTRA-AMK', '26.00'],
 ['15', 'NOV', 'AMAZON', 'SG', '177.03'],
 ['15', 'NOV', 'STARBUCKS@AMK', 'HUB(AMK)', '10.43'],
 ['15', 'NOV', 'AMAZON', 'SG', '134.59'],
 ['16', 'NOV', 'UNIQLO', 'CITY', 'SQUARE', '67.70'],
 ['16', 'NOV', 'DECATHLON', 'SG', 'LAB', '66.00'],
 ['16', 'NOV', 'BANANA', 'LEAF', '55.30'],
 ['20', 'NOV', 'ZaloraSG', '19.51CR'],
 ['20', 'NOV', 'TENDON', 'KOHAKU', '35.31'],
 ['20', 'NOV', 'GRAB*', 'S-26872624-8-462', '11.00'],
 ['20', 'NOV', "MCDONALD'S", '(HG)', '3.20'],
 ['20', 'NOV', "MCDONALD'S", '(HG)', '20.15'],
 ['22', 'NOV', "MCDONALD'S", '(SHHG)', '7.20'],
 ['23', 'NOV', 'NTUC', 'FP-HOUGANG', 'A', '11.65'],
 ['23', 'NOV', 'ESSO-CHEERS', 'BY', 'FP', '45.00'],
 ['27', 'NOV', 'ZALORASG', '95.73'],
 ['27', 'NOV', 'LAZADA', 'SINGAPORE', '55.20'],
 ['27', 'NOV', 'OMOTE', '76.03'],
 ['27', 'NOV', 'SHOPEE', '-', 'SINGAPORE', '3.8

## Create a func that does the following

Returns true if a page contains sub-total, False otherwise.
   - Don't have to process subsequent pages if previous page already contains sub-total.

In [9]:
# sub_total_regex = re.compile("SUB.TOTAL")

# def contains_sub_total(page):
#     if sub_total_regex.search(page):
#         return True
        
#     else:
#         return False
    
sub_total_regex = re.compile("SUB.TOTAL")

def contains_sub_total(page):
    if sub_total_regex.search(page):
        return True, sub_total_regex.search(page).group()
        
    else:
        return False, None

## Create another func that does the following

1. Splits string according to new lines and spaces so we can eventually load them easily into different date, txn desc, amt columns
2. Returns legitimate txns (defined as txns with dates and amt) in a new list

In [10]:
def filter_legitimate_txns(txns):
    txns_split = txns.split("\n")
    txns_split_no_ref = [txn for txn in txns_split if "Ref No." not in txn]  # to cater for UOB txns
    txns_double_split = [txn.split() for txn in txns_split_no_ref]
    
    return [txn for txn in txns_double_split if len(txn) >= 4] # Length of at least 4 (for dates, txn desc and amt)

In [11]:
first_page_legit_txns = filter_legitimate_txns(first_page_txns_raw)
first_page_legit_txns

[['12', 'NOV', 'LAZADA', 'SINGAPORE', '(PAYM', '13.39', 'C'],
 ['13', 'NOV', 'GRAB*', 'BF5684F0017318B3', '40.00'],
 ['15', 'NOV', 'FAIRPRICE', 'XTRA-AMK', '26.00'],
 ['15', 'NOV', 'AMAZON', 'SG', '177.03'],
 ['15', 'NOV', 'STARBUCKS@AMK', 'HUB(AMK)', '10.43'],
 ['15', 'NOV', 'AMAZON', 'SG', '134.59'],
 ['16', 'NOV', 'UNIQLO', 'CITY', 'SQUARE', '67.70'],
 ['16', 'NOV', 'DECATHLON', 'SG', 'LAB', '66.00'],
 ['16', 'NOV', 'BANANA', 'LEAF', '55.30'],
 ['20', 'NOV', 'ZaloraSG', '19.51CR'],
 ['20', 'NOV', 'TENDON', 'KOHAKU', '35.31'],
 ['20', 'NOV', 'GRAB*', 'S-26872624-8-462', '11.00'],
 ['20', 'NOV', "MCDONALD'S", '(HG)', '3.20'],
 ['20', 'NOV', "MCDONALD'S", '(HG)', '20.15'],
 ['22', 'NOV', "MCDONALD'S", '(SHHG)', '7.20'],
 ['23', 'NOV', 'NTUC', 'FP-HOUGANG', 'A', '11.65'],
 ['23', 'NOV', 'ESSO-CHEERS', 'BY', 'FP', '45.00'],
 ['27', 'NOV', 'ZALORASG', '95.73'],
 ['27', 'NOV', 'LAZADA', 'SINGAPORE', '55.20'],
 ['27', 'NOV', 'OMOTE', '76.03'],
 ['27', 'NOV', 'SHOPEE', '-', 'SINGAPORE', '3.8

In [12]:
print(len(first_page_legit_txns))

23


## Create another func that does the following

Reads last item in each list and does the following in a loop:
   1. Checks and keeps if numeric and contains CR, converting amt to positive
   2. If not above, then delete last item before iterating over to previous and checks 1 again

In [13]:
def process_txn_amt(txns):
    for txn in txns:
        while not txn[-1].replace(".","",1).replace(",","",1).isdigit() and not "CR" in txn[-1]:  
            txn.pop(-1)  # remove if last item in each txn is not an amt
    
        if "CR" in txn[-1]:  # if amt contains CR
            txn[-1] = txn[-1].replace("CR","",1)  # remove CR
            txn[-1] = "-" + txn[-1]  # reverse sign
            
    return txns

In [14]:
first_page_legit_txns_processed = process_txn_amt(first_page_legit_txns)
first_page_legit_txns_processed

[['12', 'NOV', 'LAZADA', 'SINGAPORE', '(PAYM', '13.39'],
 ['13', 'NOV', 'GRAB*', 'BF5684F0017318B3', '40.00'],
 ['15', 'NOV', 'FAIRPRICE', 'XTRA-AMK', '26.00'],
 ['15', 'NOV', 'AMAZON', 'SG', '177.03'],
 ['15', 'NOV', 'STARBUCKS@AMK', 'HUB(AMK)', '10.43'],
 ['15', 'NOV', 'AMAZON', 'SG', '134.59'],
 ['16', 'NOV', 'UNIQLO', 'CITY', 'SQUARE', '67.70'],
 ['16', 'NOV', 'DECATHLON', 'SG', 'LAB', '66.00'],
 ['16', 'NOV', 'BANANA', 'LEAF', '55.30'],
 ['20', 'NOV', 'ZaloraSG', '-19.51'],
 ['20', 'NOV', 'TENDON', 'KOHAKU', '35.31'],
 ['20', 'NOV', 'GRAB*', 'S-26872624-8-462', '11.00'],
 ['20', 'NOV', "MCDONALD'S", '(HG)', '3.20'],
 ['20', 'NOV', "MCDONALD'S", '(HG)', '20.15'],
 ['22', 'NOV', "MCDONALD'S", '(SHHG)', '7.20'],
 ['23', 'NOV', 'NTUC', 'FP-HOUGANG', 'A', '11.65'],
 ['23', 'NOV', 'ESSO-CHEERS', 'BY', 'FP', '45.00'],
 ['27', 'NOV', 'ZALORASG', '95.73'],
 ['27', 'NOV', 'LAZADA', 'SINGAPORE', '55.20'],
 ['27', 'NOV', 'OMOTE', '76.03'],
 ['27', 'NOV', 'SHOPEE', '-', 'SINGAPORE', '3.84'],
 

# Processing DBS second page

In [15]:
# Reads PDF's second page and extracts all txns after specified header
with pdfplumber.open(dbs_source_dir / dbs_pdf_file) as pdf:
    second_page = pdf.pages[1]
    second_page_text = second_page.extract_text()
    second_page_txns_raw_w_footnote = second_page_text.partition("2 of 3")[2]
    
    second_page_txns_raw = second_page_txns_raw_w_footnote.partition("SUB-TOTAL")[0]
    pprint.pprint(second_page_txns_raw)

('\n'
 '29 NOV SPC 1351 ANG MO KIO AV               8.50\n'
 '01 DEC GRAB* AD19385300C29104              10.00\n'
 '02 DEC WOK HEY - CHINATOWN PO               6.80\n'
 '03 DEC RAFFLES MEDICAL-HOUGAN              27.20\n'
 '04 DEC GRUB              58.85\n'
 '05 DEC FAIRPRICE XTRA-AMK              59.65\n'
 '07 DEC NTUC FP-HOUGANG A              12.20\n'
 '11 DEC GRAB* AD3DC9087F1FC0D3              10.00\n'
 '11 DEC GRAB* 8BCF36C0940DD88F              10.00\n'
 "12 DEC WATSON'S               1.60\n"
 '12 DEC CIRCL FOOD & BEVERAGE              97.37\n'
 '14 DEC 01LAZADA-DBS ONLINE IPP 06 (03)             153.16\n')


In [16]:
second_page_legit_txns = filter_legitimate_txns(second_page_txns_raw)
second_page_legit_txns

[['29', 'NOV', 'SPC', '1351', 'ANG', 'MO', 'KIO', 'AV', '8.50'],
 ['01', 'DEC', 'GRAB*', 'AD19385300C29104', '10.00'],
 ['02', 'DEC', 'WOK', 'HEY', '-', 'CHINATOWN', 'PO', '6.80'],
 ['03', 'DEC', 'RAFFLES', 'MEDICAL-HOUGAN', '27.20'],
 ['04', 'DEC', 'GRUB', '58.85'],
 ['05', 'DEC', 'FAIRPRICE', 'XTRA-AMK', '59.65'],
 ['07', 'DEC', 'NTUC', 'FP-HOUGANG', 'A', '12.20'],
 ['11', 'DEC', 'GRAB*', 'AD3DC9087F1FC0D3', '10.00'],
 ['11', 'DEC', 'GRAB*', '8BCF36C0940DD88F', '10.00'],
 ['12', 'DEC', "WATSON'S", '1.60'],
 ['12', 'DEC', 'CIRCL', 'FOOD', '&', 'BEVERAGE', '97.37'],
 ['14', 'DEC', '01LAZADA-DBS', 'ONLINE', 'IPP', '06', '(03)', '153.16']]

In [17]:
process_txn_amt(second_page_legit_txns)

[['29', 'NOV', 'SPC', '1351', 'ANG', 'MO', 'KIO', 'AV', '8.50'],
 ['01', 'DEC', 'GRAB*', 'AD19385300C29104', '10.00'],
 ['02', 'DEC', 'WOK', 'HEY', '-', 'CHINATOWN', 'PO', '6.80'],
 ['03', 'DEC', 'RAFFLES', 'MEDICAL-HOUGAN', '27.20'],
 ['04', 'DEC', 'GRUB', '58.85'],
 ['05', 'DEC', 'FAIRPRICE', 'XTRA-AMK', '59.65'],
 ['07', 'DEC', 'NTUC', 'FP-HOUGANG', 'A', '12.20'],
 ['11', 'DEC', 'GRAB*', 'AD3DC9087F1FC0D3', '10.00'],
 ['11', 'DEC', 'GRAB*', '8BCF36C0940DD88F', '10.00'],
 ['12', 'DEC', "WATSON'S", '1.60'],
 ['12', 'DEC', 'CIRCL', 'FOOD', '&', 'BEVERAGE', '97.37'],
 ['14', 'DEC', '01LAZADA-DBS', 'ONLINE', 'IPP', '06', '(03)', '153.16']]

# Putting it all together

Perform a loop that does the following:
1. Checks if page contains sub-total, as it indicates end of transactional list.
2. Extracts all transactions
3. If current page contains sub-total, do not process subsequent pages.
4. Otherwise, perform steps 1-3 again.

In [18]:
all_txns = []

In [19]:
def txn_trimming(page_text, s):
    txns_raw = page_text.partition(s)[2]
    
    sub_total_bool, sub_total_content = contains_sub_total(txns_raw)
    
    if sub_total_bool:
        return txns_raw.partition(sub_total_content)[0]
        
    else:  # if statement is UOB
        return txns_raw.partition("Pleasenote")[0]  # doesn't matter if statement is DBS as result is the same

In [20]:
with pdfplumber.open(dbs_source_dir / dbs_pdf_file) as pdf:
    for i in range(2):  # txns only extend up to 2nd page
            page_text = pdf.pages[i].extract_text()
            sub_total_bool, sub_total_content = contains_sub_total(pdf.pages[0].extract_text())
        
            if i == 0:
                txns_raw = txn_trimming(page_text, "NEW TRANSACTIONS JEROME KO JIA JIN")   
                all_txns.append(process_txn_amt(filter_legitimate_txns(txns_raw)))
        
            elif i == 1 and not sub_total_bool:  # if txns extend to 2nd page
                txns_raw = txn_trimming(page_text, "2 of 3")
                all_txns.append(process_txn_amt(filter_legitimate_txns(txns_raw)))

In [21]:
all_txns

[[['12', 'NOV', 'LAZADA', 'SINGAPORE', '(PAYM', '13.39'],
  ['13', 'NOV', 'GRAB*', 'BF5684F0017318B3', '40.00'],
  ['15', 'NOV', 'FAIRPRICE', 'XTRA-AMK', '26.00'],
  ['15', 'NOV', 'AMAZON', 'SG', '177.03'],
  ['15', 'NOV', 'STARBUCKS@AMK', 'HUB(AMK)', '10.43'],
  ['15', 'NOV', 'AMAZON', 'SG', '134.59'],
  ['16', 'NOV', 'UNIQLO', 'CITY', 'SQUARE', '67.70'],
  ['16', 'NOV', 'DECATHLON', 'SG', 'LAB', '66.00'],
  ['16', 'NOV', 'BANANA', 'LEAF', '55.30'],
  ['20', 'NOV', 'ZaloraSG', '-19.51'],
  ['20', 'NOV', 'TENDON', 'KOHAKU', '35.31'],
  ['20', 'NOV', 'GRAB*', 'S-26872624-8-462', '11.00'],
  ['20', 'NOV', "MCDONALD'S", '(HG)', '3.20'],
  ['20', 'NOV', "MCDONALD'S", '(HG)', '20.15'],
  ['22', 'NOV', "MCDONALD'S", '(SHHG)', '7.20'],
  ['23', 'NOV', 'NTUC', 'FP-HOUGANG', 'A', '11.65'],
  ['23', 'NOV', 'ESSO-CHEERS', 'BY', 'FP', '45.00'],
  ['27', 'NOV', 'ZALORASG', '95.73'],
  ['27', 'NOV', 'LAZADA', 'SINGAPORE', '55.20'],
  ['27', 'NOV', 'OMOTE', '76.03'],
  ['27', 'NOV', 'SHOPEE', '-', 'S

In [22]:
dbs_all_txns = []

# Inserting all PDFs
for folder, subfolder, pdf_files in os.walk(dbs_source_dir):
    for pdf_file in pdf_files:

        with pdfplumber.open(dbs_source_dir / pdf_file) as pdf:
            for i in range(2):  # txns only extend up to 2nd page
                    page_text = pdf.pages[i].extract_text()
                    sub_total_bool, sub_total_content = contains_sub_total(pdf.pages[0].extract_text())

                    if i == 0:
                        txns_raw = txn_trimming(page_text, "NEW TRANSACTIONS JEROME KO JIA JIN")
                        dbs_all_txns.append(process_txn_amt(filter_legitimate_txns(txns_raw)))

                    elif i == 1 and not sub_total_bool:  # if txns extend to 2nd page
                        txns_raw = txn_trimming(page_text, "2 of 3")
                        dbs_all_txns.append(process_txn_amt(filter_legitimate_txns(txns_raw)))

In [23]:
dbs_all_txns

[[['13', 'AUG', 'SHOPEE', '-', 'SINGAPORE', '5.68'],
  ['14', 'AUG', 'ISTANBLUE', 'MEZE', '-', 'MM', '26.30'],
  ['14', 'AUG', 'SULTANS', 'OF', 'SHAVE', '-', 'DU', '55.00'],
  ['20', 'AUG', 'GRAB*8C6FE4E51A3D05ACE', '20.00'],
  ['21', 'AUG', 'SHOPEE', 'SINGAPORE', '8.50'],
  ['22', 'AUG', 'GRAB*11FCC478E9A6B9708', '20.00'],
  ['25', 'AUG', 'SHAW', 'THEATRES', 'PL', 'SHAW.', '28.50'],
  ['26', 'AUG', 'GRAB*DFCA24F43BB5783AE', '20.00'],
  ['26', 'AUG', 'THE', 'DAILY', 'CUT', '-', 'ORP', '9.00'],
  ['26', 'AUG', 'STBILL', '254.32'],
  ['28', 'AUG', 'JEWEL', 'CHANGI', 'AIRPORT', '13.50'],
  ['29', 'AUG', 'AN', 'ACAI', 'AFFAIR', '-', 'JEWEL', '9.50'],
  ['30', 'AUG', 'MDM', 'LING', 'B*', 'MDMLINGBAK', '47.40'],
  ['30', 'AUG', 'SHOPEEPAY', '8.00'],
  ['01', 'SEP', 'SHOPEEPAY', '-8.00'],
  ['02', 'SEP', 'HANBAOBAO', 'PTE.', 'LTD.', '49.80'],
  ['04', 'SEP', 'KREAM', 'BEER', '42.37'],
  ['04', 'SEP', 'MUCHACHOS', '-', 'FRASERS', '34.50'],
  ['05', 'SEP', 'BAKERY', '&', 'BAR', 'ST', 'MARC', '2

# Processing UOB first page

In [24]:
# Reads PDF's first page and extracts all txns after specified header and before footnote
with pdfplumber.open(uob_source_dir / uob_pdf_file) as pdf:
    first_page = pdf.pages[0]
    first_page_text = first_page.extract_text()
    first_page_txns_raw = first_page_text.partition("PREVIOUS BALANCE")[2].partition("Pleasenote")[0]
    print(first_page_txns_raw)

 1,549.38
04 AUG  04 AUG  PAYMT THRU E-BANK/HOMEB/CYBERB (EP12) 1,549.38CR
11 AUG  08 AUG  CR LATE CHARGE 100.00CR
12 AUG  11 AUG  UOB YOLO Cash Rebate 36.60CR
18 JUL  17 JUL  JOO SINGAPORE 124.75
Ref No. : 74541830200288070619671
21 JUL  21 JUL  KKH-SELF PAYMENT KIOSK 19SINGAPORE 254.66
Ref No. : 74541830203288071877788
27 JUL  25 JUL  TWO MEN BAGELS- MM SINGAPORE 29.02
Ref No. : 74541830207288074054506
30 JUL  29 JUL  THE DAILY CUT - ORP SINGAPORE 9.00
Ref No. : 74508980211009239158760
01 AUG  30 JUL  NTUC FP - CHINATOWN PT SINGAPORE 11.80
Ref No. : 74508980212009243375144
03 AUG  03 AUG  LATE CHARGES 100.00
04 AUG  03 AUG  OVERLIMIT FEE 40.00
11 AUG  09 AUG  Hanbaobao Pte. Ltd. SINGAPORE 35.30
Ref No. : 74508980223009294955916
11 AUG  09 AUG  GOMO MOBILE PLAN SINGAPORE 30.00
Ref No. : 74541830224288081355259
12 AUG  12 AUG  INTERESTS 45.27 



In [25]:
# Looks at whitespace characters in text
print(repr(first_page_txns_raw))

' 1,549.38\n04 AUG  04 AUG  PAYMT THRU E-BANK/HOMEB/CYBERB (EP12) 1,549.38CR\n11 AUG  08 AUG  CR LATE CHARGE 100.00CR\n12 AUG  11 AUG  UOB YOLO Cash Rebate 36.60CR\n18 JUL  17 JUL  JOO SINGAPORE 124.75\nRef No. : 74541830200288070619671\n21 JUL  21 JUL  KKH-SELF PAYMENT KIOSK 19SINGAPORE 254.66\nRef No. : 74541830203288071877788\n27 JUL  25 JUL  TWO MEN BAGELS- MM SINGAPORE 29.02\nRef No. : 74541830207288074054506\n30 JUL  29 JUL  THE DAILY CUT - ORP SINGAPORE 9.00\nRef No. : 74508980211009239158760\n01 AUG  30 JUL  NTUC FP - CHINATOWN PT SINGAPORE 11.80\nRef No. : 74508980212009243375144\n03 AUG  03 AUG  LATE CHARGES 100.00\n04 AUG  03 AUG  OVERLIMIT FEE 40.00\n11 AUG  09 AUG  Hanbaobao Pte. Ltd. SINGAPORE 35.30\nRef No. : 74508980223009294955916\n11 AUG  09 AUG  GOMO MOBILE PLAN SINGAPORE 30.00\nRef No. : 74541830224288081355259\n12 AUG  12 AUG  INTERESTS 45.27 \n'


In [26]:
# Split by new lines
first_page_txns_raw.split("\n")

[' 1,549.38',
 '04 AUG  04 AUG  PAYMT THRU E-BANK/HOMEB/CYBERB (EP12) 1,549.38CR',
 '11 AUG  08 AUG  CR LATE CHARGE 100.00CR',
 '12 AUG  11 AUG  UOB YOLO Cash Rebate 36.60CR',
 '18 JUL  17 JUL  JOO SINGAPORE 124.75',
 'Ref No. : 74541830200288070619671',
 '21 JUL  21 JUL  KKH-SELF PAYMENT KIOSK 19SINGAPORE 254.66',
 'Ref No. : 74541830203288071877788',
 '27 JUL  25 JUL  TWO MEN BAGELS- MM SINGAPORE 29.02',
 'Ref No. : 74541830207288074054506',
 '30 JUL  29 JUL  THE DAILY CUT - ORP SINGAPORE 9.00',
 'Ref No. : 74508980211009239158760',
 '01 AUG  30 JUL  NTUC FP - CHINATOWN PT SINGAPORE 11.80',
 'Ref No. : 74508980212009243375144',
 '03 AUG  03 AUG  LATE CHARGES 100.00',
 '04 AUG  03 AUG  OVERLIMIT FEE 40.00',
 '11 AUG  09 AUG  Hanbaobao Pte. Ltd. SINGAPORE 35.30',
 'Ref No. : 74508980223009294955916',
 '11 AUG  09 AUG  GOMO MOBILE PLAN SINGAPORE 30.00',
 'Ref No. : 74541830224288081355259',
 '12 AUG  12 AUG  INTERESTS 45.27 ',
 '']

UOB e-statements have an additional element; ref no. are embedded within transactional listing. Hence, there's a need to edit *filter_legitimate_txns* function to incorporate this.

In [27]:
# After editing, the function then also works well for UOB pages
pprint.pprint(filter_legitimate_txns(first_page_txns_raw))

[['04',
  'AUG',
  '04',
  'AUG',
  'PAYMT',
  'THRU',
  'E-BANK/HOMEB/CYBERB',
  '(EP12)',
  '1,549.38CR'],
 ['11', 'AUG', '08', 'AUG', 'CR', 'LATE', 'CHARGE', '100.00CR'],
 ['12', 'AUG', '11', 'AUG', 'UOB', 'YOLO', 'Cash', 'Rebate', '36.60CR'],
 ['18', 'JUL', '17', 'JUL', 'JOO', 'SINGAPORE', '124.75'],
 ['21',
  'JUL',
  '21',
  'JUL',
  'KKH-SELF',
  'PAYMENT',
  'KIOSK',
  '19SINGAPORE',
  '254.66'],
 ['27',
  'JUL',
  '25',
  'JUL',
  'TWO',
  'MEN',
  'BAGELS-',
  'MM',
  'SINGAPORE',
  '29.02'],
 ['30',
  'JUL',
  '29',
  'JUL',
  'THE',
  'DAILY',
  'CUT',
  '-',
  'ORP',
  'SINGAPORE',
  '9.00'],
 ['01',
  'AUG',
  '30',
  'JUL',
  'NTUC',
  'FP',
  '-',
  'CHINATOWN',
  'PT',
  'SINGAPORE',
  '11.80'],
 ['03', 'AUG', '03', 'AUG', 'LATE', 'CHARGES', '100.00'],
 ['04', 'AUG', '03', 'AUG', 'OVERLIMIT', 'FEE', '40.00'],
 ['11', 'AUG', '09', 'AUG', 'Hanbaobao', 'Pte.', 'Ltd.', 'SINGAPORE', '35.30'],
 ['11', 'AUG', '09', 'AUG', 'GOMO', 'MOBILE', 'PLAN', 'SINGAPORE', '30.00'],
 ['12

In [28]:
# Removing post dates
first_page_txns_processed = filter_legitimate_txns(first_page_txns_raw)
for txn in first_page_txns_processed:
    del txn[0:2]

pprint.pprint(first_page_txns_processed)

[['04', 'AUG', 'PAYMT', 'THRU', 'E-BANK/HOMEB/CYBERB', '(EP12)', '1,549.38CR'],
 ['08', 'AUG', 'CR', 'LATE', 'CHARGE', '100.00CR'],
 ['11', 'AUG', 'UOB', 'YOLO', 'Cash', 'Rebate', '36.60CR'],
 ['17', 'JUL', 'JOO', 'SINGAPORE', '124.75'],
 ['21', 'JUL', 'KKH-SELF', 'PAYMENT', 'KIOSK', '19SINGAPORE', '254.66'],
 ['25', 'JUL', 'TWO', 'MEN', 'BAGELS-', 'MM', 'SINGAPORE', '29.02'],
 ['29', 'JUL', 'THE', 'DAILY', 'CUT', '-', 'ORP', 'SINGAPORE', '9.00'],
 ['30', 'JUL', 'NTUC', 'FP', '-', 'CHINATOWN', 'PT', 'SINGAPORE', '11.80'],
 ['03', 'AUG', 'LATE', 'CHARGES', '100.00'],
 ['03', 'AUG', 'OVERLIMIT', 'FEE', '40.00'],
 ['09', 'AUG', 'Hanbaobao', 'Pte.', 'Ltd.', 'SINGAPORE', '35.30'],
 ['09', 'AUG', 'GOMO', 'MOBILE', 'PLAN', 'SINGAPORE', '30.00'],
 ['12', 'AUG', 'INTERESTS', '45.27']]


UOB also appends "CR" at the end of transactional amt if it's a rebate, thus *process_txn_amt* function also works in this case.

In [29]:
process_txn_amt(filter_legitimate_txns(first_page_txns_raw))

[['04',
  'AUG',
  '04',
  'AUG',
  'PAYMT',
  'THRU',
  'E-BANK/HOMEB/CYBERB',
  '(EP12)',
  '-1,549.38'],
 ['11', 'AUG', '08', 'AUG', 'CR', 'LATE', 'CHARGE', '-100.00'],
 ['12', 'AUG', '11', 'AUG', 'UOB', 'YOLO', 'Cash', 'Rebate', '-36.60'],
 ['18', 'JUL', '17', 'JUL', 'JOO', 'SINGAPORE', '124.75'],
 ['21',
  'JUL',
  '21',
  'JUL',
  'KKH-SELF',
  'PAYMENT',
  'KIOSK',
  '19SINGAPORE',
  '254.66'],
 ['27',
  'JUL',
  '25',
  'JUL',
  'TWO',
  'MEN',
  'BAGELS-',
  'MM',
  'SINGAPORE',
  '29.02'],
 ['30',
  'JUL',
  '29',
  'JUL',
  'THE',
  'DAILY',
  'CUT',
  '-',
  'ORP',
  'SINGAPORE',
  '9.00'],
 ['01',
  'AUG',
  '30',
  'JUL',
  'NTUC',
  'FP',
  '-',
  'CHINATOWN',
  'PT',
  'SINGAPORE',
  '11.80'],
 ['03', 'AUG', '03', 'AUG', 'LATE', 'CHARGES', '100.00'],
 ['04', 'AUG', '03', 'AUG', 'OVERLIMIT', 'FEE', '40.00'],
 ['11', 'AUG', '09', 'AUG', 'Hanbaobao', 'Pte.', 'Ltd.', 'SINGAPORE', '35.30'],
 ['11', 'AUG', '09', 'AUG', 'GOMO', 'MOBILE', 'PLAN', 'SINGAPORE', '30.00'],
 ['12', 

# Processing UOB second page

In [30]:
# Reads PDF's first page and extracts all txns after specified header and before footnote
with pdfplumber.open(uob_source_dir / uob_pdf_file) as pdf:
    second_page = pdf.pages[1]
    second_page_text = second_page.extract_text()
    second_page_txns_raw = second_page_text.partition("Date Date SGD")[2].partition("SUB TOTAL")[0]
    print(second_page_txns_raw)

In [31]:
# Testing robustness of filter_legitimate_txns function by running it on second page as well
pprint.pprint(filter_legitimate_txns(second_page_txns_raw))

[]


In [32]:
process_txn_amt(filter_legitimate_txns(second_page_txns_raw))

[]

# Putting it all together

In [33]:
# Reads PDF's first page and extracts all txns after specified header and before footnote
with pdfplumber.open(uob_source_dir / uob_pdf_file) as pdf:
    first_page = pdf.pages[0]
    first_page_text = first_page.extract_text()
#     first_page_txns_raw = first_page_text.partition("PREVIOUS BALANCE")[2].partition("Pleasenote")[0]
#     print(first_page_txns_raw)
    print(txn_trimming(first_page_text,"PREVIOUS BALANCE"))

 1,549.38
04 AUG  04 AUG  PAYMT THRU E-BANK/HOMEB/CYBERB (EP12) 1,549.38CR
11 AUG  08 AUG  CR LATE CHARGE 100.00CR
12 AUG  11 AUG  UOB YOLO Cash Rebate 36.60CR
18 JUL  17 JUL  JOO SINGAPORE 124.75
Ref No. : 74541830200288070619671
21 JUL  21 JUL  KKH-SELF PAYMENT KIOSK 19SINGAPORE 254.66
Ref No. : 74541830203288071877788
27 JUL  25 JUL  TWO MEN BAGELS- MM SINGAPORE 29.02
Ref No. : 74541830207288074054506
30 JUL  29 JUL  THE DAILY CUT - ORP SINGAPORE 9.00
Ref No. : 74508980211009239158760
01 AUG  30 JUL  NTUC FP - CHINATOWN PT SINGAPORE 11.80
Ref No. : 74508980212009243375144
03 AUG  03 AUG  LATE CHARGES 100.00
04 AUG  03 AUG  OVERLIMIT FEE 40.00
11 AUG  09 AUG  Hanbaobao Pte. Ltd. SINGAPORE 35.30
Ref No. : 74508980223009294955916
11 AUG  09 AUG  GOMO MOBILE PLAN SINGAPORE 30.00
Ref No. : 74541830224288081355259
12 AUG  12 AUG  INTERESTS 45.27 



In [34]:
uob_all_txns = []

# Inserting all PDFs
for folder, subfolder, pdf_files in os.walk(uob_source_dir):
    for pdf_file in pdf_files:

        with pdfplumber.open(uob_source_dir / pdf_file) as pdf:
            for i in range(2):  # txns only extend up to 2nd page
                    page_text = pdf.pages[i].extract_text()
                    sub_total_bool, sub_total_content = contains_sub_total(pdf.pages[0].extract_text())

                    if i == 0:
                        txns_raw = txn_trimming(page_text, "PREVIOUS BALANCE")
                        uob_all_txns.append(process_txn_amt(filter_legitimate_txns(txns_raw)))

                    elif i == 1 and not sub_total_bool:  # if txns extend to 2nd page
                        txns_raw = txn_trimming(page_text, "Date Date SGD")
                        uob_all_txns.append(process_txn_amt(filter_legitimate_txns(txns_raw)))
                        
for monthly_txns in uob_all_txns:
    for txn in monthly_txns:
        del txn[0:2]  # remove post dates

In [35]:
uob_all_txns

[[['04', 'AUG', 'PAYMT', 'THRU', 'E-BANK/HOMEB/CYBERB', '(EP12)', '-1,549.38'],
  ['08', 'AUG', 'CR', 'LATE', 'CHARGE', '-100.00'],
  ['11', 'AUG', 'UOB', 'YOLO', 'Cash', 'Rebate', '-36.60'],
  ['17', 'JUL', 'JOO', 'SINGAPORE', '124.75'],
  ['21', 'JUL', 'KKH-SELF', 'PAYMENT', 'KIOSK', '19SINGAPORE', '254.66'],
  ['25', 'JUL', 'TWO', 'MEN', 'BAGELS-', 'MM', 'SINGAPORE', '29.02'],
  ['29', 'JUL', 'THE', 'DAILY', 'CUT', '-', 'ORP', 'SINGAPORE', '9.00'],
  ['30', 'JUL', 'NTUC', 'FP', '-', 'CHINATOWN', 'PT', 'SINGAPORE', '11.80'],
  ['03', 'AUG', 'LATE', 'CHARGES', '100.00'],
  ['03', 'AUG', 'OVERLIMIT', 'FEE', '40.00'],
  ['09', 'AUG', 'Hanbaobao', 'Pte.', 'Ltd.', 'SINGAPORE', '35.30'],
  ['09', 'AUG', 'GOMO', 'MOBILE', 'PLAN', 'SINGAPORE', '30.00'],
  ['12', 'AUG', 'INTERESTS', '45.27']],
 [],
 [['19', 'AUG', 'PAYMT', 'THRU', 'E-BANK/HOMEB/CYBERB', '(EP11)', '-543.20'],
  ['01', 'SEP', 'CR', 'INTEREST', '-45.27'],
  ['01', 'SEP', 'CR', 'OVERLIMIT', 'FEE', '-40.00'],
  ['11', 'SEP', 'UOB'

In [36]:
all_txns = dbs_all_txns.copy()
all_txns.extend(uob_all_txns)

In [37]:
all_txns

[[['13', 'AUG', 'SHOPEE', '-', 'SINGAPORE', '5.68'],
  ['14', 'AUG', 'ISTANBLUE', 'MEZE', '-', 'MM', '26.30'],
  ['14', 'AUG', 'SULTANS', 'OF', 'SHAVE', '-', 'DU', '55.00'],
  ['20', 'AUG', 'GRAB*8C6FE4E51A3D05ACE', '20.00'],
  ['21', 'AUG', 'SHOPEE', 'SINGAPORE', '8.50'],
  ['22', 'AUG', 'GRAB*11FCC478E9A6B9708', '20.00'],
  ['25', 'AUG', 'SHAW', 'THEATRES', 'PL', 'SHAW.', '28.50'],
  ['26', 'AUG', 'GRAB*DFCA24F43BB5783AE', '20.00'],
  ['26', 'AUG', 'THE', 'DAILY', 'CUT', '-', 'ORP', '9.00'],
  ['26', 'AUG', 'STBILL', '254.32'],
  ['28', 'AUG', 'JEWEL', 'CHANGI', 'AIRPORT', '13.50'],
  ['29', 'AUG', 'AN', 'ACAI', 'AFFAIR', '-', 'JEWEL', '9.50'],
  ['30', 'AUG', 'MDM', 'LING', 'B*', 'MDMLINGBAK', '47.40'],
  ['30', 'AUG', 'SHOPEEPAY', '8.00'],
  ['01', 'SEP', 'SHOPEEPAY', '-8.00'],
  ['02', 'SEP', 'HANBAOBAO', 'PTE.', 'LTD.', '49.80'],
  ['04', 'SEP', 'KREAM', 'BEER', '42.37'],
  ['04', 'SEP', 'MUCHACHOS', '-', 'FRASERS', '34.50'],
  ['05', 'SEP', 'BAKERY', '&', 'BAR', 'ST', 'MARC', '2

# Categorizing transactions

In [38]:
# Represent txns according to dates, desc and amt
for monthly_txns in all_txns:
    for txn in monthly_txns:
        print(" ".join(txn[0:2]), " ".join(txn[2:len(txn)-1]) , txn[-1])

13 AUG SHOPEE - SINGAPORE 5.68
14 AUG ISTANBLUE MEZE - MM 26.30
14 AUG SULTANS OF SHAVE - DU 55.00
20 AUG GRAB*8C6FE4E51A3D05ACE 20.00
21 AUG SHOPEE SINGAPORE 8.50
22 AUG GRAB*11FCC478E9A6B9708 20.00
25 AUG SHAW THEATRES PL SHAW. 28.50
26 AUG GRAB*DFCA24F43BB5783AE 20.00
26 AUG THE DAILY CUT - ORP 9.00
26 AUG STBILL 254.32
28 AUG JEWEL CHANGI AIRPORT 13.50
29 AUG AN ACAI AFFAIR - JEWEL 9.50
30 AUG MDM LING B* MDMLINGBAK 47.40
30 AUG SHOPEEPAY 8.00
01 SEP SHOPEEPAY -8.00
02 SEP HANBAOBAO PTE. LTD. 49.80
04 SEP KREAM BEER 42.37
04 SEP MUCHACHOS - FRASERS 34.50
05 SEP BAKERY & BAR ST MARC 25.41
08 SEP SWEE HENG BAKERY-CL7 1.50
09 SEP SWEE HENG BAKERY-CL7 1.80
10 SEP YARDBIRD 140.06
12 SEP FINEST FUNAN 2.00
15 JUL GRAB*487A744E79FC5A74C 50.00
15 JUL GV DBO 25.00
17 JUL SHOPEE - SINGAPORE 12.49
21 JUL GRAB*F434B4F4AB82070D4 30.00
26 JUL GET IT BY CHANGI RECOM 55.99
30 JUL GRAB*6BA2D47E3AB10455A 30.00
31 JUL PIEDRA NEGRA 48.02
01 AUG THE BEAN HOPPERS PTE L 13.90
02 AUG LIBERTY WIRELESS PTE L

Understanding double for loops in list comprehensions: https://stackoverflow.com/questions/17657720/python-list-comprehension-double-for.

[A for B in C for D in E]
- A is the item in the resulting list
- D is each item in collection E

In [39]:
categorized_txns = [{"Date": " ".join(txn[0:2]), "Txn Desc": " ".join(txn[2:len(txn)-1]), "Amt": txn[-1]} for monthly_txns in all_txns for txn in monthly_txns]
categorized_txns

[{'Date': '13 AUG', 'Txn Desc': 'SHOPEE - SINGAPORE', 'Amt': '5.68'},
 {'Date': '14 AUG', 'Txn Desc': 'ISTANBLUE MEZE - MM', 'Amt': '26.30'},
 {'Date': '14 AUG', 'Txn Desc': 'SULTANS OF SHAVE - DU', 'Amt': '55.00'},
 {'Date': '20 AUG', 'Txn Desc': 'GRAB*8C6FE4E51A3D05ACE', 'Amt': '20.00'},
 {'Date': '21 AUG', 'Txn Desc': 'SHOPEE SINGAPORE', 'Amt': '8.50'},
 {'Date': '22 AUG', 'Txn Desc': 'GRAB*11FCC478E9A6B9708', 'Amt': '20.00'},
 {'Date': '25 AUG', 'Txn Desc': 'SHAW THEATRES PL SHAW.', 'Amt': '28.50'},
 {'Date': '26 AUG', 'Txn Desc': 'GRAB*DFCA24F43BB5783AE', 'Amt': '20.00'},
 {'Date': '26 AUG', 'Txn Desc': 'THE DAILY CUT - ORP', 'Amt': '9.00'},
 {'Date': '26 AUG', 'Txn Desc': 'STBILL', 'Amt': '254.32'},
 {'Date': '28 AUG', 'Txn Desc': 'JEWEL CHANGI AIRPORT', 'Amt': '13.50'},
 {'Date': '29 AUG', 'Txn Desc': 'AN ACAI AFFAIR - JEWEL', 'Amt': '9.50'},
 {'Date': '30 AUG', 'Txn Desc': 'MDM LING B* MDMLINGBAK', 'Amt': '47.40'},
 {'Date': '30 AUG', 'Txn Desc': 'SHOPEEPAY', 'Amt': '8.00'},
 {

In [40]:
df_categorized_txns = pd.DataFrame(categorized_txns)
df_categorized_txns

,Date,Txn Desc,Amt
0,13 AUG,SHOPEE - SINGAPORE,5.68
1,14 AUG,ISTANBLUE MEZE - MM,26.30
2,14 AUG,SULTANS OF SHAVE - DU,55.00
3,20 AUG,GRAB*8C6FE4E51A3D05ACE,20.00
4,21 AUG,SHOPEE SINGAPORE,8.50
...,...,...,...
352,04 JAN,MUJI @ ION ORCHARD SINGAPORE,8.20
353,06 JAN,UNIQLO TIONG BAHRU PLAZA SINGAPORE,29.90
354,06 JAN,TONGUE TIP LZ BEEF NOODLESINGAPORE,19.05
355,08 JAN,FIVE GUYS SINGAPORE,51.00


In [41]:
df_categorized_txns["Date"] = df_categorized_txns["Date"] + " 2020"
df_categorized_txns

,Date,Txn Desc,Amt
0,13 AUG 2020,SHOPEE - SINGAPORE,5.68
1,14 AUG 2020,ISTANBLUE MEZE - MM,26.30
2,14 AUG 2020,SULTANS OF SHAVE - DU,55.00
3,20 AUG 2020,GRAB*8C6FE4E51A3D05ACE,20.00
4,21 AUG 2020,SHOPEE SINGAPORE,8.50
...,...,...,...
352,04 JAN 2020,MUJI @ ION ORCHARD SINGAPORE,8.20
353,06 JAN 2020,UNIQLO TIONG BAHRU PLAZA SINGAPORE,29.90
354,06 JAN 2020,TONGUE TIP LZ BEEF NOODLESINGAPORE,19.05
355,08 JAN 2020,FIVE GUYS SINGAPORE,51.00


In [42]:
# Categorizing txns
shopping = re.compile(r'''.*Shopee(pay)?.*
                      |.*Lazada.*
                      |.*Zalora.*
                      |.*Uniqlo.*
                      |.*Asos.*
                      |.*Ghbass.*
                      |.*Decathlon.*
                      |.*Amazon.*
                      |.*Watson.*
                      |.*Guardian.*''', re.I | re.VERBOSE)
utilities = re.compile(r".*Liberty Wireless.*|.*Gomo.*", re.I)
holiday = re.compile(r".*Agoda*.|.*Scoot.*|.*Hotel.*", re.I)
grooming = re.compile(".*Sultans of shave.*", re.I)
entertainment = re.compile(r".*GV.*|.*Shaw.*", re.I)
others = re.compile(r".* Fee.*|.*Charge.*|.*Interest.*|.*Bank.*|.*Rebate.*", re.I)

In [45]:
def categorize_txns(s):
    if shopping.search(s["Txn Desc"]):  # None if there's no match
        return "Shopping"
    
    elif utilities.search(s["Txn Desc"]):
        return "Utilities"
    
    elif holiday.search(s["Txn Desc"]):
        return "Holiday"
    
    elif grooming.search(s["Txn Desc"]):
        return "Grooming"
    
    elif entertainment.search(s["Txn Desc"]):
        return "Entertainment"
    
    elif others.search(s["Txn Desc"]):
        return "Others"
    
    elif s["Date"] == "27 JUN 2020":
        return "Birthday"
    
    elif s["Date"] == "19 MAR 2020":
        return "Anniversary"
    
    else:
        return "Food"

In [46]:
df_categorized_txns["Category"] = df_categorized_txns.apply(categorize_txns, axis=1)

In [49]:
df_categorized_txns[df_categorized_txns["Date"] == "19 MAR 2020"]

,Date,Txn Desc,Amt,Category


In [50]:
df_categorized_txns.to_csv(dest_csv / "2020 transactions.csv")